In [305]:
# General Imports
import pandas as pd
import numpy as np
import re
import string
import spacy

# Model Specific imports
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn import metrics


In [13]:
# Run the following if autocompletion is not working
%config Completer.use_jedi = False

In [238]:
# PARAMETERS:
train_on_clean = True # Change to get the other data.

In [293]:
file_name = 'train_clean.csv'
if not train_on_clean:
    file_name = 'train.csv'

df = pd.read_csv(file_name)
df = df[df.text.notnull()]
df = df[df.keyword.notnull()]
print(f'Imported {file_name}')    

Imported train_clean.csv


In [294]:
import nltk
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon') ## this only needs to be run once

vader_model = SentimentIntensityAnalyzer()

In [295]:
# Sentiments for text
df['text_compound'] = df['text'].apply(lambda x:vader_model.polarity_scores(x)['compound'])

# Sentiments for keyword
df['keyword_compound'] = df['keyword'].apply(lambda x:vader_model.polarity_scores(f"We have {x}")['compound'])

In [296]:

features_prod = lambda x: (x.text_compound * x.keyword_compound if 
    np.logical_and(x.text_compound!=0, x.keyword_compound!=0) else 
    min(x.text_compound, x.keyword_compound))

df['features_compound'] = df.apply(features_prod, axis=1)

In [297]:
df.sample(frac=1)

,keyword,location,text,target,compound,text_compound,keyword_compound,features_compound
596,bleeding,NaN,cute dinner date til cam nose start bleed,0,-0.3570,0.4588,0.0000,0.000000
767,bloody,"Adelaide, South Australia",bloody sexy drool,0,0.1280,0.1280,-0.4404,-0.056371
5647,suicide%20bomber,NaN,talk risk suicide bomber hiding migrant stow a...,1,-0.8720,-0.8271,0.0000,-0.827100
3179,famine,"Kyiv, Ukraine",russia food crematoria provoke outrage country...,1,-0.8519,-0.8519,0.0000,-0.851900
1978,deluge,Australia,wa smile july deluge west australian,0,0.4767,0.3612,0.0000,0.000000
...,...,...,...,...,...,...,...,...
653,blight,Daruka (near Tamworth) NSW,load welfare love sponge blight society,0,0.5994,0.6369,0.0000,0.000000
5471,smoke,NaN,smoke blunt amp ciggs,0,-0.2960,0.0000,0.0000,0.000000
199,annihilation,Republic of Texas,annihilation jeb christie amp kasich hour away...,0,0.6486,0.4588,0.0000,0.000000
720,blood,"???????, ??'??????",blood group ve associate gastric carcinoma say...,0,0.0000,0.0000,0.0000,0.000000


In [298]:
x_compound = df['compound'].apply(lambda x: 0 if x >= 0 else 1).values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(
    x_compound, df['target'], test_size=0.2, random_state=1)

In [299]:
clf = MultinomialNB()

clf.fit(X_train, y_train)

prediction = clf.predict(X_test)

print(metrics.accuracy_score(y_test, prediction))


0.5713218820014937


In [302]:
x_compound = df['features_compound'].apply(lambda x: 0 if x >= 0 else 1).values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(
    x_compound, df['target'], test_size=0.4, random_state=1)

In [307]:
clf = LinearSVC()

clf.fit(X_train, y_train)

prediction = clf.predict(X_test)

print(metrics.accuracy_score(y_test, prediction))


0.6075429424943988
